# Fragmentation Gain Intensity Analysis per Municipalities

This notebook presents town- based forest fragmentation change intensity in Massachusetts. My poster presented hwo forest fragmentation changed overtime in Massachusetts. However, there is no information to answer questions like "where are clusters with high forest fragmentation intensities?" Hence, this dashboard employs local Moran's I that identifies spatial clusters and outliers that has significantly high or low values. 

### Methods 
I downloaded [NOAA's coastal land cover dataset](https://coast.noaa.gov/digitalcoast/data/ccapregional.html), and reclassified 2006 and 2016 layer into forest or non- forest. Then, the forest pixels are reclassified if a pixel belongs to fragmented forest or non- fragmented forest. The "fragmented forest pixels" is those classified into rare, patchy, or transitional of [foregrond area density analysis of GuidoToolbox](https://ies-ows.jrc.ec.europa.eu/gtb/GTB/psheets/GTB-Fragmentation-FADFOS.pdf), as a comparison to American's black bears' minimum home range (24km\^2). Then, I calculated persistence and changes of the pixels at each time point, and calculated

1.  Area of fragmented, non- fragmented, and non- forest as a ratio to towns' area.
2.  Gain, losses, and change intensities of the fragmentation categories.

-   Gain intensity = \# of pixels of gains/ \# of pixels of interest at 2016
-   Loss intensity = \# of pixels of losses/ \# of pixels of interest at 2006
-   Change intensity = \# of pixels of changes/ \# of total pixels in the town

3.  Quantity labels that represents if the towns experienced net gain or net loss, as well as quantity intensities.
  - Gain quantity intensity = # of pixels of gain quantity/ # of pixels of interest at 2016
  - Loss intensity = # of pixels of losses/ # of pixels of pixels of interest at 2006
  - Difference Quantity = # of quantities/ # of pixels in each municipality
4.  Local Moran's I on the change intensity showe high/ high cluster in east- central Mass

In [1]:
import rasterio as rio 
import geopandas as gpd 
import numpy as np 
import pandas as pd

In [3]:
import os
os.chdir("/Volumes/volume/GIS Projects/Research/byTown")

## Step0

### Add Municipality ID

In [4]:
# READ DATA####
v = gpd.read_file("./townssurvey_shp/TOWNSSURVEY_POLY.shp")

# ADD ID COLUMN####
v["townID"] = range(1, len(v) + 1)

# EXPORT THE DATA####
# v.to_file("town_with_id.shp")

### Rasterize Polygon with ID
This section rasterize polygon of municipality layers to calculate intensities by each town.

In [5]:
def rasterizeVctWVal(vctPath: str,
                     rstPath: str,
                     valColumn: str,
                     outPath: str):
    v = gpd.read_file(vctPath)
    r = rio.open(rstPath)

    # Create List containing Geometry and Value====
    geomval = ((geom, val) for geom, val in zip(v.geometry, v[valColumn]))

    # Rasterize Polygon====
    from rasterio.features import rasterize
    outr = rasterize(geomval,
                     out_shape = r.shape,
                     transform = r.transform,
                     all_touched = True,
                     fill = 0,
                     dtype = np.uint16)
    # Write Raster====
    with rio.open(
        outPath, "w",
        driver = "GTiff",
        crs = r.crs,
        transform = r.transform,
        dtype = np.uint16,
        count = 1,
        width = r.width,
        height = r.height) as newr:
        newr.write(outr, indexes = 1)


In [6]:
# rasterizeVctWVal("town_with_id.shp",
#                  "fad2006_4conn_all.tif",
#                  "townID",
#                  "towns.tif")

## Step2

### Assign values representing gain(1), loss(2), presence persistence(3), and absence persistence(4).
This section creates 3 layers: which are fragmentation forest change, non- fragmentation forest change, and non- forest change, containing the 4 values (+ out of boundary)

In [7]:
def reclassifyChanges(xPath,
                      yPath,
                      targetVal,
                      outPath,
                      outOfBoundaryVal = 0):
    # Read Raster====
    xarr = rio.open(xPath).read(1)
    yarr = rio.open(yPath).read(1)

    outarr = xarr.copy()

    # Reclassify Raster====
    outarr[(xarr != targetVal) & (yarr == targetVal)] = 1
    outarr[(xarr == targetVal) & (yarr != targetVal)] = 2
    outarr[(xarr == targetVal) & (yarr == targetVal)] = 3
    outarr[(xarr != targetVal) & (yarr != targetVal)] = 4

    # Assign 0 to out of boundary----
    outarr[(xarr == outOfBoundaryVal) | (yarr == outOfBoundaryVal)] = 0


    # Write Raster with Updating Meta Data====
    ref = rio.open(xPath)
    outMeta = ref.meta.copy()
    outMeta.update({
        "dirver": "GTiff",
        "height": ref.height,
        "width": ref.width,
        "transform": ref.transform,
        "nodata": 0,
        "dtype": np.uint8,
        "crs": ref.crs
    })

    with rio.open(outPath, "w", **outMeta) as newr:
        newr.write(outarr, 1)

In [8]:
# for i in range(1, 4):
#     reclassifyChanges("fad2006_4conn_all.tif",
#                       "fad2016_4conn_all.tif",
#                       i,
#                       f"fad_changes_{i}.tif")

## Step3

### Count Foreground, Background, and Extent Size for Each Municipality ID at each time point
This section aggregates the number of presence persistence, absence persistence, gain, and loss by town.

In [9]:
def countPixels(forestRasterPath, 
                municipalRaster,
                valueOfInterest):
    # Open Files and Remove Out of Boundary====
    fr_original = rio.open(forestRasterPath).read(1)
    mr_original = rio.open(municipalRaster).read(1)
    fr = fr_original[(mr_original != 0) & (fr_original != 0)].flatten()
    # print(np.unique(fr))
    mr = mr_original[(mr_original != 0) & (fr_original != 0)].flatten()

    # Create Array of 0 or 1: Returns 1 If Object of Interest====
    fr_for = np.zeros(np.size(fr)).astype(np.uint8)
    fr_for[(fr == valueOfInterest)] = 1
    
    print(np.unique(fr_for))

    # Construct Data Frame with a Value for Every Pixel====
    colname1 = "fore"

    df = pd.DataFrame({
        "muni": mr,
        colname1: fr_for,
        "Extent": 1
    })

    # Aggregate the Data Frame by Municipality ID====
    df_out = df.groupby(by = ["muni"]).sum()

    # Calculate Background Size and Foreground Intensity====
    df_out[f"bac_{valueOfInterest}"] = (df_out["Extent"] - df_out[colname1]).astype(np.uint32)
    df_out[f"fore_intensity"] = (df_out[colname1]/ df_out["Extent"])* 100

    return df_out


In [10]:
# Count Fragmented Forest at 2006====
# frag2006 = countPixels("fad2006_4conn_all.tif",
#                        "towns.tif",
#                        1)
# frag2006

In [11]:
# Count Non Fragmented Forest at 2006====
# nonfrag2006 = countPixels("fad2006_4conn_all.tif",
#                           "towns.tif",
#                           2)
# nonfrag2006

In [12]:
# Count Non Forest at 2006====
# nonforest2006 = countPixels("fad2006_4conn_all.tif",
#                             "towns.tif",
#                             3)
# nonforest2006

In [13]:
# Count Classes at 2016====
# frag2016 = countPixels("fad2016_4conn_all.tif",
#                        "towns.tif",
#                        1)
# nonfrag2016 = countPixels("fad2016_4conn_all.tif",
#                        "towns.tif",
#                        2)
# nonforest2016 = countPixels("fad2016_4conn_all.tif",
#                             "towns.tif",
#                             3)

As a result, we created 3 layers that contains foreground area density (= number of pixels of interest/ number of pixels within a municipality).
Note that files will be exported later.

[Foreground Intensity](./imgs/screenshot2024-03-1121.23.50.png)

### Sum up the G, L, Pp, Ap by Each Municipality
This section returns data frame that includes gain and losses of each category.

In [14]:
def aggregateByMuni(forestRasterPath,
                    municipalRaster,
                    valueOfInterest):
    
    colName = "val"
    # Open File and Remove Out of Boundary====
    fr = rio.open(forestRasterPath).read(1)
    mr = rio.open(municipalRaster).read(1)
    fr2 = fr[(mr != 0) & (fr != 0)].flatten()
    mr2 = mr[(mr != 0) & (fr != 0)].flatten()

    # Check A Pixel Holds the Value of Interest: eg, Assign 1 If The Interest is Gain====
    fr_for = np.zeros(np.size(fr2)).astype(np.uint8)
    fr_for[(fr2 == valueOfInterest)] = 1

    # Return Summarized Data Frame by Municipality====
    df = pd.DataFrame({
        "muni": mr2,
        colName: fr_for
    })
    df_out = df.groupby(by = ["muni"]).sum()

    return df_out

In [15]:
# Count Pixel of Fragmented Forest Gain====
# frag_gain = aggregateByMuni("fad_changes_1.tif",
#                             "towns.tif",
#                             1)


In [16]:
# frag_loss = aggregateByMuni("fad_changes_1.tif",
#                             "towns.tif",
#                             2)

In [17]:
# # Calculate Fagmentation Persistence====
# frag_pp = aggregateByMuni("fad_changes_1.tif",
#                           "towns.tif",
#                           3)

In [18]:
# # Calculate Fragmented Forest Absence Persistence====
# frag_ap = aggregateByMuni("fad_changes_1.tif",
#                           "towns.tif",
#                           4)

In [19]:
# nonfrag_gain = aggregateByMuni("fad_changes_2.tif",
#                                "towns.tif",
#                                1)
# nonfrag_loss = aggregateByMuni("fad_changes_2.tif",
#                                "towns.tif",
#                                2)
# nonfrag_pp = aggregateByMuni("fad_changes_2.tif",
#                              "towns.tif",
#                              3)
# nonfrag_ap = aggregateByMuni("fad_changes_2.tif",
#                              "towns.tif",
#                              4)

In [ ]:
# nonfor_gain = aggregateByMuni("fad_changes_3.tif",
#                                "towns.tif",
#                                1)
# nonfor_loss = aggregateByMuni("fad_changes_3.tif",
#                                "towns.tif",
#                                2)
# nonfor_pp = aggregateByMuni("fad_changes_3.tif",
#                              "towns.tif",
#                              3)
# nonfor_ap = aggregateByMuni("fad_changes_3.tif",
#                             "towns.tif",
#                             4)

### Calculate the Intensity
This section calculates metrics, especially below.
- Quantity label: Showing net gain, net loss, or no net change
- Loss Intensity = Loss/ Forest size at 2006
- Gain Intensity = Gain/ Forest size at 2016
- Change Intensity = Change Size (= Loss + Gain)/ Extent size at 2016

In [20]:
def calcChangeIntensityByMuni(gain_df,
                              loss_df,
                              size_df_2006,
                              size_df_2016):
    # Extract Values from Data Frame and Calculate Metrics====
    gain = gain_df.val.to_numpy()
    for_extent2016 = size_df_2016.fore.to_numpy()
    gain_intensity = (gain/ for_extent2016)* 100
    loss = loss_df.val.to_numpy()
    extent_size = size_df_2006.Extent.to_numpy()
    for_extent2006 = size_df_2006.fore.to_numpy()
    loss_intensity = (loss/ for_extent2006)* 100

    # Calculate Quantity====
    quantity = gain.astype(np.int64) - loss.astype(np.int64)
    quantity_abs = np.absolute(quantity)

    # Label Quantity: If Gain is Larger than Loss, then Label "Gain" and so on====
    quantity_label = np.where(quantity == 0,
                              "zero",
                              np.where(quantity < 0,
                                       "loss",
                                       "gain"))
    gain_quantity = np.where(quantity_label == "gain",
                             quantity_abs,
                             0)
    loss_quantity = np.where(quantity_label == "loss",
                             quantity_abs,
                             0)
    
    # Calculate Category Exchange====
    category_exchange = np.where(quantity <= 0,
                                 (loss - quantity_abs)* 2,
                                 (gain - quantity_abs)* 2)
    # Divide the Exchange by 2 to Avoid Double Count====
    exchange_gain_loss = category_exchange/ 2
    change = exchange_gain_loss + quantity_abs
    change_intensity = (change/ extent_size)* 100

    # Return the Summarized Data Frame====
    out_df = pd.DataFrame({
        "muni": gain_df.index,
        "fore_int16": size_df_2016.fore_intensity,
        "fore_int06": size_df_2006.fore_intensity,
        "gain": gain,
        "loss": loss,
        "gain_int": gain_intensity,
        "loss_int": loss_intensity,
        "extent": extent_size,
        "gain_quantity": gain_quantity,
        "loss_quantity": loss_quantity,
        "quant_lab": quantity_label,
        "catg_xchg": category_exchange,
        "xchg_gl": exchange_gain_loss,
        "change": change,
        "change_int": change_intensity
    })

    # out_df.fillna(0).to_csv("test.csv")
    return out_df.fillna(0)
    

In [21]:
# frag_changes = calcChangeIntensityByMuni(frag_gain,
#                                          frag_loss,
#                                          frag2006,
#                                          frag2016)

# non_frag_changes = calcChangeIntensityByMuni(nonfrag_gain,
#                                              nonfrag_loss,
#                                              nonfrag2006,
#                                              nonfrag2016)

# non_forest_changes = calcChangeIntensityByMuni(nonfor_gain,
#                                                nonfor_loss,
#                                                nonforest2006,
#                                                nonforest2016)


## Step4
### Join the results and the original vector file
This section exports the file as shape file.

In [22]:
v = v[["townID", "geometry"]]

In [23]:
# frag_changes = frag_changes.drop("muni", axis = 1).reset_index()

# gdf_frag = v.merge(frag_changes, how = "inner", left_on = "townID", right_on = "muni")
# gdf_frag.to_file("./output/frag_changes.shp")

In [24]:
# non_frag_changes = non_frag_changes.drop("muni", axis = 1).reset_index()

# gdf_nonfrag = v.merge(non_frag_changes, how = "inner", left_on = "townID", right_on = "muni")
# gdf_nonfrag.to_file("./output/nonfrag_changes.shp")

In [25]:
# non_forest_changes = non_forest_changes.drop("muni", axis = 1).reset_index()

# gdf_nonfor = v.merge(non_forest_changes, how = "inner", left_on = "townID", right_on = "muni")
# gdf_nonfor.to_file("./output/nonfor_changes.shp")

As a result, we got several images

[Quantity Label](imgs/screenshot2024-03-1121.35.50.png)
[Intensities of Fragmented Forest](imgs/screenshot2024-03-1121.36.15.png)
[Intensities of Non- fragmented forest](imgs/screenshot2024-03-1121.51.02.png)
[Intensities of Non- forest](imgs/screenshot2024-03-1121.36.48.png)

## Step 5

### Calculate Local Morans I
The local moran's I identifies cluster of significantly high values/ low values, as well as spatial outliers.

In [26]:
def calcMoran(gdf,
              variables_list,
              out_name):
    import pygeoda 

    # Open File and Create a Spatial Weight File====
    file = pygeoda.open(gdf)
    w = pygeoda.queen_weights(file)

    # Calculate Local Moran and Get P Values and Label Category Value====
    d = {}
    for i in range(len(variables_list)):
        lm = pygeoda.local_moran(w, file[variables_list[i]])
        pval = pd.Series(lm.lisa_pvalues())
        lab = pd.Series(lm.lisa_clusters())
        p_key = f"p_{variables_list[i]}"
        l_key = f"l_{variables_list[i]}"
        d[p_key] = pval
        d[l_key] = lab

    # Combine with Geometry and Export as Shape File====
    d["geometry"] = gdf["geometry"]

    out_gdf = gpd.GeoDataFrame(d)
    out_gdf.to_file(out_name)


In [27]:
# calcMoran(gdf_frag, ["gain_int", "loss_int", "change_int"], "./output/frag_int_moran.shp")

In [28]:
# calcMoran(gdf_nonfrag, ["gain_int", "loss_int", "change_int"], "./output/nonfrag_int_moran.shp")
# calcMoran(gdf_nonfor, ["gain_int", "loss_int", "change_int"], "./output/nonfor_int_moran.shp")

As a result, we got an image.

[Image of Local Moran's I](imgs/screenshot2024-03-1121.51.51.png)

## Result & Discussion
We found that
1.  Fragmented forest is frequent in eastern MA, but non- fragmente forest is frequent in western MA.
2.  Nonetheless, non- fragmented forest experienced net loss in the eastern MA, and gained in central MA
    - There is spatial patterns of distributions of fragmented forest and changes.
3.  Non- fragmented forest experienced net loss in most of the towns.
4.  Non- forest experienced net gain in most of the towns.
    - Many towns experienced forest fragmentation and losses during the time interval.
5. Local Moran's I on the change intensity showe high/ high cluster in east- central Mass
    - This indicates that *significantly large change intensities are around Worcester, not around Boston!*

## Appendix
### Key Reference
-  Rogan, J., Wright, T. M., Cardille, J., Pearsall, H., Ogneva-Himmelberger, Y., Riemann, R., ... & Partington, K. (2016). Forest fragmentation in Massachusetts, USA: A town-level assessment using Morphological spatial pattern analysis and affinity propagation. GIScience & Remote Sensing, 53(4), 506-519.
-  Pontius Jr, R. G. (2022). Metrics that make a difference. Springer Nature Switzerland AG: Cham, Switzerland.

### Software
-   Vogt P. and Riitters K. (2017). GuidosToolbox: universal digital image object analysis. European Journal of Remote Sensing, 50, 1, pp. 352-361, doi: 10.1080/22797254.2017.1330650
-   Li X. and Anselin L (2019) pygeoda 0.0.8. GeoDaCenter. <https://github.com/GeoDaCenter/pygeoda>
-   Python Software Foundation (2023) Python 3.11.5. Webpage of Python <https://www.python.org/downloads/release/python-3115/>

*I would like to thank Professor John Rogan for his constructive feedbacks.*

## Next Steps
1. Explain quantity and exchange (gross change vs net change)